In [8]:
# From https://idiotdeveloper.com/unet-segmentation-with-pretrained-mobilenetv2-as-encoder/
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import sys
sys.path.insert(0,'../')

from src.utils.kerasDataLoader import DataGenerator

import numpy as np
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision

from tensorflow.keras import backend as K
import numpy as np



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

def strong_aug(p=0.5):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)


In [16]:
def make_model(image_size, n_classes = 21):
    print((*image_size, 3))
    inputs = Input(shape=(*image_size, 3), name="input_image")
    
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.35)
    skip_connection_names = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    encoder_output = encoder.get_layer("block_13_expand_relu").output
    
    f = [16, 32, 48, 64]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        x_skip = encoder.get_layer(skip_connection_names[-i]).output
        x = UpSampling2D((2, 2))(x)
        x = Concatenate()([x, x_skip])
        
        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
    x = Conv2D(n_classes, (1, 1), padding="same")(x)
    x = Activation("softmax")(x)
    
    model = Model(inputs, x)
    return model

In [17]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [18]:
model = make_model(image_size=(224,224))

opt = tf.keras.optimizers.Adam(0.001)

metrics = [dice_coef, Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

(224, 224, 3)


In [6]:
train_dataset = DataGenerator(batch_size=16, augmentation=strong_aug)
valid_dataset = DataGenerator(step='valid', shuffle=False)

In [ ]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
41/41 [==============================] - 82s 2s/step - loss: 0.9011 - dice_coef: 0.0989 - recall_2: 0.1277 - precision_2: 0.6287 - val_loss: 0.9333 - val_dice_coef: 0.0667 - val_recall_2: 0.1147 - val_precision_2: 0.3342
Epoch 2/20
41/41 [==============================] - 82s 2s/step - loss: 0.8257 - dice_coef: 0.1743 - recall_2: 0.2855 - precision_2: 0.5207 - val_loss: 0.8972 - val_dice_coef: 0.1028 - val_recall_2: 0.1992 - val_precision_2: 0.1800
Epoch 3/20
41/41 [==============================] - 82s 2s/step - loss: 0.7760 - dice_coef: 0.2240 - recall_2: 0.4114 - precision_2: 0.5331 - val_loss: 0.9165 - val_dice_coef: 0.0835 - val_recall_2: 0.1725 - val_precision_2: 0.1842
Epoch 4/20
41/41 [==============================] - 80s 2s/step - loss: 0.7421 - dice_coef: 0.2579 - recall_2: 0.4708 - precision_2: 0.5339 - val_loss: 0.8932 - val_dice_coef: 0.1068 - val_recall_2: 0.2103 - val_precision_2: 0.1668
Epoch 5/20
41/41 [==============================] - 81s 2s/step - loss: 